In [7]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings

In [8]:
read_dir = '../data/processed/'
train_data = pd.read_excel(read_dir + 'train_data.xlsx')
test_data = pd.read_excel(read_dir + 'test_data.xlsx')

In [9]:
warnings.simplefilter('ignore')

print("--- 1. Đang gộp dữ liệu Train + Test cho Prophet CV ---")
full_data = pd.concat([train_data, test_data])

prophet_df = full_data[['datetime', 'temp']].copy()
prophet_df.rename(columns={'datetime': 'ds', 'temp': 'y'}, inplace=True)
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

print(f"Tổng số dữ liệu: {len(prophet_df)} ngày")

model_prophet = Prophet(
    seasonality_mode='multiplicative'
)

model_prophet.fit(prophet_df)

df_cv = cross_validation(
    model_prophet,
    initial=f'{len(train_data)} days',
    period='90 days',              
    horizon='5 days',         
    parallel="processes" 
)

df_p = performance_metrics(df_cv)

metrics_prophet = df_p[['horizon', 'mse', 'mae']].copy()
metrics_prophet['horizon'] = metrics_prophet['horizon'].dt.days

print("Kết quả Cross-Validation của Prophet:")
print(metrics_prophet)

try:
    mae_t1 = metrics_prophet[metrics_prophet['horizon'] == 1]['mae'].values[0]
    mae_t2 = metrics_prophet[metrics_prophet['horizon'] == 2]['mae'].values[0]
    mae_t3 = metrics_prophet[metrics_prophet['horizon'] == 3]['mae'].values[0]
    mae_t4 = metrics_prophet[metrics_prophet['horizon'] == 4]['mae'].values[0]
    mae_t5 = metrics_prophet[metrics_prophet['horizon'] == 5]['mae'].values[0]
    avg_mae_prophet = metrics_prophet['mae'].mean()

    print(f"Prophet (t+1): MAE={mae_t1:.3f}")
    print(f"Prophet (t+2): MAE={mae_t2:.3f}")
    print(f"Prophet (t+3): MAE={mae_t3:.3f}")
    print(f"Prophet (t+4): MAE={mae_t4:.3f}")
    print(f"Prophet (t+5): MAE={mae_t5:.3f}")
    print(f"Average MAE: {avg_mae_prophet:.3f}")
except Exception as e:
    print(f"Không thể trích xuất MAE tự động, vui lòng xem bảng 'metrics_prophet' ở trên. Lỗi: {e}")

--- 1. Đang gộp dữ liệu Train + Test cho Prophet CV ---
Tổng số dữ liệu: 3920 ngày


14:04:29 - cmdstanpy - INFO - Chain [1] start processing
14:04:29 - cmdstanpy - INFO - Chain [1] done processing


Kết quả Cross-Validation của Prophet:
   horizon       mse       mae
0        1  8.845634  2.256659
1        2  5.085443  1.910226
2        3  4.835574  1.673349
3        4  5.086273  1.824721
4        5  2.740199  1.345488
Prophet (t+1): MAE=2.257
Prophet (t+2): MAE=1.910
Prophet (t+3): MAE=1.673
Prophet (t+4): MAE=1.825
Prophet (t+5): MAE=1.345
Average MAE: 1.802
